In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import operator
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# import warnings
import warnings
# ignore warnings
warnings.filterwarnings("ignore")
from subprocess import check_output
from IPython.display import Markdown as md
from prettytable import PrettyTable
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from numpy import sqrt
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Data Pre-Processing

In [ ]:
# Membaca data dari dataset
data = pd.read_csv('../input/vertebralcolumndataset/column_2C.csv')

In [ ]:
# Mendapatkan dummies values
df = pd.get_dummies(data)
df.head(10)

In [ ]:
# Menghapus fitur atau kolom yang tidak diperlukan
df.drop("class_Normal",axis = 1, inplace = True) 
df.head(10)

In [ ]:
# Melihat info pada data
data.info()

In [ ]:
# Melihat deskripsi pada data
data.describe()

Eksplorasi Data

In [ ]:
color_list = ['red' if i=='Abnormal' else 'green' for i in data.loc[:,'class']]
pd.plotting.scatter_matrix(data.loc[:, data.columns != 'class'],
                                       c=color_list,
                                       figsize= [15,15],
                                       diagonal='hist',
                                       alpha=0.5,
                                       s = 200,
                                       marker = '*',
                                       edgecolor= "black")
plt.show()

In [ ]:
# Mengetahui nilai pada class abnormal dan normal
sns.countplot(x="class", data=data)
data.loc[:,'class'].value_counts()

> ******REGRESSION**

In [ ]:
# Membuat data1 termasuk pelvic_incidence sebagai fitur and sacral_slope variabel target
data1 = data[data['class'] =='Abnormal']
x = np.array(data1.loc[:,'pelvic_incidence']).reshape(-1,1)
y = np.array(data1.loc[:,'sacral_slope']).reshape(-1,1)
# Gambar Scatter Plot
plt.figure(figsize=[10,10])
plt.scatter(x=x,y=y)
plt.xlabel('pelvic_incidence')
plt.ylabel('sacral_slope')
plt.show()

In [ ]:
# Import LinearRegression dari sklearn.linear_model
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
predict_space = np.linspace(min(x), max(x)).reshape(-1,1)
# Fit
reg.fit(x,y)
# Prediksi
predicted = reg.predict(predict_space)
# # R^2 
# print('R^2 score: ',reg.score(x, y))
degree = 1

x = np.array(data1.loc[:,'pelvic_incidence']).reshape(-1,1)
y = np.array(data1.loc[:,'sacral_slope']).reshape(-1,1)
polynomial_features= PolynomialFeatures(degree=degree)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)


# tabel parameter
tabel_parameter = PrettyTable(['parameter', 'nilai'])
tabel_parameter.add_row(['Polynomial\nDegree', degree])
tabel_parameter.add_row(['RMSE','{:.10}'.format(rmse)])
tabel_parameter.add_row(['R^2', '{:.10}'.format(r2)])

# Gambar Regresi linear and scatter plot
plt.plot(predict_space, predicted, color='black', linewidth=3)
plt.scatter(x=x,y=y)
plt.title('Biomechanical features of orthopedic patients\n')
plt.xlabel('pelvic_incidence')
plt.ylabel('sacral_slope')
plt.show()
print(tabel_parameter)

In [ ]:
# Mengetahui hasil persamaan pada nilai y 
def display_equation(model):
    for coef in model.coef_:
        pass
    streq = "$y = " +str(model.intercept_[0])
    for i,c in enumerate(coef):
        j = len(coef)-i-1
        if abs(c) > c:
            sign = "-"
        else:
            sign = "+"
        if i > 1:
            streq += sign +str(abs(c)) + " \cdot x^{"+str(i)+"}"
        elif i == 1:
            streq += sign +str(abs(c)) + " \cdot x"

    streq =   streq + "$"
    return md(streq)
display_equation(model)

Model Polynomial Regression (Degree = 10)

In [ ]:
# Polinomial
degree = 10

x = np.array(data1.loc[:,'pelvic_incidence']).reshape(-1,1)
y = np.array(data1.loc[:,'sacral_slope']).reshape(-1,1)
polynomial_features= PolynomialFeatures(degree=degree)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)

In [ ]:
# Tabel parameter
tabel_parameter = PrettyTable(['parameter', 'nilai'])
tabel_parameter.add_row(['Polynomial\nDegree', degree])
tabel_parameter.add_row(['RMSE','{:.10}'.format(rmse)])
tabel_parameter.add_row(['R^2', '{:.10}'.format(r2)])

In [ ]:
# Gambar scatter plot 
plt.scatter(x, y, s=10)
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.title('Biomechanical features of orthopedic patients\n')
plt.xlabel('\npelvic_incidence')
plt.ylabel('sacral_slope')

plt.plot(x, y_poly_pred, color='m')
plt.show()
print(tabel_parameter)

In [ ]:
# Mengetahui hasil persamaan pada nilai y 
def display_equation(model):
    for coef in model.coef_:
        pass
    streq = "$y = " +str(model.intercept_[0])
    for i,c in enumerate(coef):
        j = len(coef)-i-1
        if abs(c) > c:
            sign = "-"
        else:
            sign = "+"
        if i > 1:
            streq += sign +str(abs(c)) + " \cdot x^{"+str(i)+"}"
        elif i == 1:
            streq += sign +str(abs(c)) + " \cdot x"

    streq =   streq + "$"
    return md(streq)
display_equation(model)

KMEANS CLUSTERING

In [ ]:
# Membaca data dari dataset
data = pd.read_csv('../input/vertebralcolumndataset/column_2C.csv')
# Mendapatkan dummies values 
df = pd.get_dummies(data)
df.head(10)

In [ ]:
# Menghapus satu fitur atau kolom yang tidak diperlukan
df.drop("class_Normal",axis = 1, inplace = True) 
df.head(10)

In [ ]:
# Melihat info pada data frame
df.info()

In [ ]:
# Melihat class Abnormal
X = df

y = df['class_Abnormal']

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X['class_Abnormal'] = le.fit_transform(X['class_Abnormal'])

y = le.transform(y)

In [ ]:
# Melihat info pada data
X.info()

In [ ]:
# Melihat 5 data teratas
X.head()

In [ ]:
# Menambahkan kolom
cols = X.columns

In [ ]:
# Import MinMaxScaler dari sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()

X = ms.fit_transform(X)

In [ ]:
# Menambahkan kolom pada data frame
X = pd.DataFrame(X, columns=[cols])

In [ ]:
# Melihat 5 data teratas yang sudah ditambahkan kolom baru
X.head()

In [ ]:
# Import library
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

from yellowbrick.cluster import KElbowVisualizer

# Buat dataset dengan 3 kluster acak
X, y = make_blobs(n_samples=310, n_features=10, centers=3, random_state=0)

# Instantiate model clustering dan visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,11))

visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
# Tidak ada label di dalam data
plt.scatter(data['pelvic_radius'],data['degree_spondylolisthesis'])
plt.xlabel('pelvic_radius')
plt.ylabel('degree_spondylolisthesis')
plt.show()

In [ ]:
# Nilai K = 2
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2,random_state=0)

kmeans.fit(X)

labels = kmeans.labels_

# Periksa berapa banyak sampel yang diberi label dengan benar

correct_labels = sum(y == labels)

print("Result: %d out of %d samples were correctly labeled." % (correct_labels, y.size))

print('Accuracy score: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
# K = 2 ( Nilai Sembarang )
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0) 

kmeans.fit(X)

In [ ]:
# Mengetahui nilai array pada KMeans
kmeans.cluster_centers_

In [ ]:
# Mengetahui nilai inersia pada KMeans
kmeans.inertia_

In [ ]:
# Nilai K = 3
kmeans = KMeans(n_clusters=3, random_state=0)

kmeans.fit(X)

# Periksa berapa banyak sampel yang diberi label dengan benar
labels = kmeans.labels_

correct_labels = sum(y == labels)
print("Result: %d out of %d samples were correctly labeled." % (correct_labels, y.size))
print('Accuracy score: {0:0.2f}'. format(correct_labels/float(y.size)))

In [ ]:
# K = 3 ( Nilai Optimal )
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0) 

kmeans.fit(X)

In [ ]:
# Mengetahui nilai array pada KMeans
kmeans.cluster_centers_

In [ ]:
# Mengetahui nilai inersia pada KMeans
kmeans.inertia_

In [ ]:
# Gambar Plot dari nilai K = 2
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=2,cluster_std=0.60, shuffle=True, random_state=0)
plt.scatter(X[:, 0], X[:, 1], edgecolor='blue', s=50);

In [ ]:
# Import data dari library
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=2, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [ ]:
# Gambar scatter plot dengan  klaster
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=50, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=50, c='orange',
    marker='o', edgecolor='black',
    label='cluster 2'
)

# Mengetahui sentroid pada scatter plot
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1)
plt.grid()
plt.show()

In [ ]:
# Gambar Plot dari nilai K = 3
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=3,cluster_std=0.60, shuffle=True, random_state=0)
plt.scatter(X[:, 0], X[:, 1], edgecolor='blue', s=50);

In [ ]:
# Import data dari library
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=3, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [ ]:
# Gambar plot dengan 3 klaster
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=50, c='lightgreen',
    marker='s', edgecolor='black',
    label='cluster 1'
)

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=50, c='orange',
    marker='o', edgecolor='black',
    label='cluster 2'
)

plt.scatter(
    X[y_km == 2, 0], X[y_km == 2, 1],
    s=50, c='lightblue',
    marker='v', edgecolor='black',
    label='cluster 3'
)


# Mengetahui sentroid pada scatter plot
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=250, marker='*',
    c='red', edgecolor='black',
    label='centroids'
)
plt.legend(scatterpoints=1)
plt.grid()
plt.show()

Silhouette Plot of KMeans

In [ ]:
# K = 2 ( Nilai Sembarang )  
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import KMeans
model = SilhouetteVisualizer(KMeans(n_clusters=2))
model.fit(X)
model.show()

In [ ]:
# K= 3 ( Nilai Optimal )
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.cluster import KMeans
model = SilhouetteVisualizer(KMeans(n_clusters=3))
model.fit(X)
model.show()

In [ ]:
# Import data dari library
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
n_clusters = 2
# Inisialisasi clusterer dengan nilai n_clusters dan generator acak
clusterer = KMeans(n_clusters=n_clusters, random_state=10)
cluster_labels = clusterer.fit_predict(X)
# Mengetahui banyaknya klaster dan nilai rata-rata silhouette 
silhouette_avg = silhouette_score(X, cluster_labels)
print("For n_clusters =", n_clusters,
      "The average silhouette_score is :", silhouette_avg)
# Hitung skor silhouette untuk setiap sampel 
sample_silhouette_values = silhouette_samples(X, cluster_labels)

In [ ]:
n_clusters = 3
# Inisialisasi clusterer dengan nilai n_clusters dan generator acak 
clusterer = KMeans(n_clusters=n_clusters, random_state=10)
cluster_labels = clusterer.fit_predict(X)
# Mengetahui banyaknya klaster dan nilai rata-rata silhouette 
silhouette_avg = silhouette_score(X, cluster_labels)
print("For n_clusters =", n_clusters,
      "The average silhouette_score is :", silhouette_avg)
# Hitung skor siluet untuk setiap sampel 
sample_silhouette_values = silhouette_samples(X, cluster_labels)